## Scrapping

This notebook is used to scrap [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M] and obtain the list of postal codes of canada

In [1]:
import pandas as pd

from urllib import request
from bs4 import BeautifulSoup

Load webpage and obtain html

In [2]:
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = request.urlopen(html).read()

soup = BeautifulSoup(source, "html.parser")
table = soup.find('table', attrs={'class':'wikitable'})
table_rows = table.find_all('tr')

get table elements from html

In [3]:
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

In [4]:
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


remove cases where borough is not assigned

In [5]:
df = df[df.Borough != "Not assigned"]

join various neighbourhoods in the same borough

In [6]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

replcae not assigned neighbourhoods with borough name

In [7]:
def replace(r):
    if r['Neighborhood'] == "Not assigned": r["Neighborhood"] = r["Borough"]
    return r

In [8]:
df = df.apply(replace,axis=1)

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df.shape

(103, 3)